In [ ]:
!pip install selenium
!pip install pandas
!pip install requests
#!pip install 
!pip install beautifulsoup4

In [1]:
import os
#from glob import glob
import time
from selenium import webdriver
import urllib.request
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import requests
from bs4 import BeautifulSoup
import csv

import pandas as pd

# links
# login_address='https://app.business-in-a-box.com/account/login'
login_address = 'http://energybill.ir/Account/Login'
# target_page='https://app.business-in-a-box.com/doc/business-plan-template-D12528'
# target_xml_doc='https://templates.biztree.com/svgs/docviewerWebApp.html?v4#12528.xml'
# one_sample_svg = 'https://templates.biztree.com/svgs/12528_files/page1.svg'

In [2]:
#Definning some basic functions for later usage
def clickOnId(id):
	browser.find_element_by_id(id).click()
	
def clickOnXpath(xpath):
	browser.find_element_by_xpath(xpath).click()

def clickOnClass(class_name):
	browser.find_element(By.CLASS_NAME, class_name).click()

def TypeInId(id,toBeTyped):
	elems = browser.find_elements_by_id(id)
	elems[0].send_keys(toBeTyped)
	
def TypeInXpath(xpath,toBeTyped):
	elems = browser.find_elements_by_xpath(xpath)
	elems[0].send_keys(toBeTyped)
    
def openBrowser():
    chromeOptions = webdriver.ChromeOptions()
    chromedriver = "chromedriver.exe"
    global browser
    browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=chromeOptions)
    #browser.fullscreen_window()	
    return browser

def login_into_system():
    browser.get(login_address)
    TypeInXpath('//*[@id="Email"]',"")
    TypeInXpath('//*[@id="Password"]',"")
    browser.find_element(By.XPATH,'/html/body/div[1]/div[2]/div/form/div[2]/div[2]/button').click()
#     clickOnClass('btn btn-loginBlue')
#     clickOnXpath('//*[@id="BibWebApp"]/div[2]/div[2]/div/div[2]/div/div/form/button')
    
    time.sleep(3)
    

In [20]:
browser=openBrowser()
login_into_system()

C:\Users\TW\AppData\Local\Temp/ipykernel_7348/184779882.py:23: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=chromeOptions)


In [4]:
def get_source_code(url):
    browser.get(url)
    elem = browser.find_element(By.XPATH,"//*")
    source_code = elem.get_attribute("outerHTML")
    return source_code

In [5]:
# class="navbar-text navbar-text-title paging-info navbar-pager"

def get_num_pages(url):
#     browser.get(url)
#     if browser.find_element(By.XPATH,'//*[@id="pageNum"]') is None:
#         elem = browser.find_element(By.XPATH,'//*[@id="pageNum"]')
#     else:
#         elem = browser.find_element(By.CLASS_NAME,"navbar-text navbar-text-title paging-info navbar-pager")
#     return elem

    source_code = get_source_code(url)
    soup = BeautifulSoup(source_code, 'html.parser')
    return int(soup.find("li", {"class": "navbar-text navbar-text-title paging-info navbar-pager"}).text.strip())

In [13]:
def write_in_csv(vahed,page,html_page):
    
    soup_page = BeautifulSoup(html_page, 'html.parser')
    parent_dir = os.getcwd()
    directory = '{}'.format(vahed)
    path = os.path.join(parent_dir, directory)
    
    os.makedirs(path, exist_ok=True)

    outfile = open("{}\\table_data{}_{}.csv".format(vahed,vahed,page),"w",newline='',encoding="utf-8")
    writer = csv.writer(outfile)

    #tree = BeautifulSoup(html_document,"lxml")
    table_tag = soup_page.find_all("table")[0]
    tab_data = [[item.text for item in row_data.select("th,td")]
                    for row_data in table_tag.select("tr")]

    if tab_data is not None:
      for data in tab_data:
          writer.writerow(data)
          #print(' '.join(data))
    try:    
        pd.read_csv("{}//table_data{}_{}.csv".format(vahed,vahed,page)).to_excel('{}//table_data{}_{}.xlsx'.format(vahed,vahed,page))
    except:
        print('no file found, vahed:{}, page:{}'.format(vahed,page))
#     workbook = xlsxwriter.Workbook('"table_data{}_{}.csv".format(vahed)')
#     worksheet = workbook.add_worksheet()

In [7]:
main_url = 'http://energybill.ir/Meter/Index/Electrical'
#url_page = 'http://energybill.ir/Meter{}/Reading/Page{}'.format(vahed,page)

In [24]:
list_vahed = [
    #5513,5518,5523,5557,
#     5558,5559,5560,5561,5562,5563,5564,5565,5593,5594,5603,
#     5604,5605,5606,5607,5608,5609,5637,5638,5639,5640,5641,
#     5642,5643,5644,5645,5514,5519,5524,5526,5527,5595,5596,
#     5597,5598,5599,5600,5601,5602,5646,5647,5648,5654,5664,
#     5665,5670,5672,5673,5674,5675,5676,5677,5678,5679,5680,
#     10277,5516,5525,5532,5533,5534,5535,5536,5537,5538,5550,
#     5551,5566,5567,5568,5649,5650,5651,5652,5653,5660,5662,
#     5663,5666,5667,5668,5669,5683,5684,5685,10278,5511,5528,
#     5529,5530,5531,5589,5590,5591,5592,5655,5656,5657,5658,
#     5659,5661,5681,5682,5686,5687,5688,5689,5690,5691,5692,
#     5693,5694,5695,5696,5697,5698,5515,5520,5521,5522,5575,
#     5576,5586,5587,5588,5610,5611,5612,5613,5614,5615,5616,
#     5617,5618,5625,5626,5627,5628,5629,5630,5631,5632,5633,
    #5634,5635,
    5636,5517,5541,5545,5546,5547,5549,5525,5553,
    5554,5555,5556,5569,5570,5571,5572,5573,5574,5578,5579,
    5580,5581,5582,5583,5619,5620,5621,5622,5623,5624,5671,
    5542,5543,5544,5548,5577,5584,5585,10647,10648,10649]

In [25]:

def run_this():
    for vahed in list_vahed:
        write_in_csv(vahed,1,get_source_code('http://energybill.ir/Meter{}/Reading/Page{}'.format(vahed,1))) 
        num_page = get_num_pages('http://energybill.ir/Meter{}/Reading/Page{}'.format(vahed,1))

        for page in range(2,num_page+1):
            write_in_csv(vahed,page,get_source_code('http://energybill.ir/Meter{}/Reading/Page{}'.format(vahed,page))) 

In [26]:
run_this()

no file found, vahed:5541, page:207
no file found, vahed:5545, page:204
no file found, vahed:5525, page:224
no file found, vahed:5553, page:228
no file found, vahed:5570, page:221
no file found, vahed:5574, page:221
no file found, vahed:5579, page:216
no file found, vahed:5580, page:215
no file found, vahed:5581, page:214
no file found, vahed:5583, page:213
no file found, vahed:5619, page:221
no file found, vahed:5623, page:218
no file found, vahed:5671, page:254
no file found, vahed:5542, page:207
no file found, vahed:5543, page:206
no file found, vahed:5544, page:205
no file found, vahed:5577, page:218
no file found, vahed:5584, page:221
no file found, vahed:10648, page:209


no file found, vahed:5558, page:228
no file found, vahed:5560, page:227
no file found, vahed:5563, page:225
no file found, vahed:5603, page:226
no file found, vahed:5604, page:226
no file found, vahed:5605, page:225
no file found, vahed:5607, page:224
no file found, vahed:5637, page:230
no file found, vahed:5643, page:226
no file found, vahed:5514, page:232
no file found, vahed:5519, page:230
no file found, vahed:5524, page:233
no file found, vahed:5595, page:1472
no file found, vahed:5596, page:1468
no file found, vahed:5600, page:1467
no file found, vahed:5646, page:231
no file found, vahed:5648, page:229
no file found, vahed:5672, page:230
no file found, vahed:5679, page:227
no file found, vahed:5532, page:222
no file found, vahed:5533, page:222
no file found, vahed:5534, page:221
no file found, vahed:5550, page:229
no file found, vahed:5566, page:224
no file found, vahed:5567, page:222
no file found, vahed:5568, page:222
no file found, vahed:5649, page:229
no file found, vahed:5651, page:228
no file found, vahed:5653, page:226
no file found, vahed:5662, page:217
no file found, vahed:5663, page:209
no file found, vahed:5667, page:254
no file found, vahed:5668, page:254
no file found, vahed:5685, page:221
no file found, vahed:5511, page:226
no file found, vahed:5528, page:225
no file found, vahed:5530, page:224
no file found, vahed:5589, page:215
no file found, vahed:5592, page:213
no file found, vahed:5655, page:222
no file found, vahed:5657, page:220
no file found, vahed:5658, page:220
no file found, vahed:5681, page:224
no file found, vahed:5682, page:222
no file found, vahed:5693, page:221
no file found, vahed:5697, page:218
no file found, vahed:5515, page:222
no file found, vahed:5520, page:220
no file found, vahed:5613, page:216
no file found, vahed:5614, page:216
no file found, vahed:5616, page:215
no file found, vahed:5617, page:215
no file found, vahed:5618, page:215
no file found, vahed:5631, page:208
no file found, vahed:5632, page:207

no file found, vahed:5557, page:230


'C:\\Users\\Romina\\Desktop'